# COVID Incremental Ventilator Requirements

In [1]:
import sys
import pandas as pd
sys.path.append('../src')
from restart import NoteCompose
from data import Data
from util import set_config, to_df, to_sheet, display_population, format_population

## Methodology Overview and Assumptions (for Rich to Vet)

Quick overview of the current state of this document: the notebook code here walks you through the methodology and assumptions made, and then the very last cell is an `ipyvuetify` implementation of everything so it hopefully looks pretty. That might be a good place for you to put the final documentation. Also, if you run `voila` on this notebook with the template `vuetify-default` (you might need to `pip install voila-vuetify`), then it won't display any of the stuff not in that cell. 

Code to fetch the IHME and MIT numbers is in `src/extern/data/epidemiological` (written by Ethan). He has mentioned that the MIT projections have a peak in August and are monotically decreasing ever since, so maybe we should drop those. 

In addition, various RAND and CAN scenarios were pulled manually directly from the CalCat website and placed into the model, using the upper bounds. Some of these should probably be pruned out, as (by design) they are unrealistically high and then we're also taking an upper bound on top of that. We use the hack of representing these demands in our population module. 

Also, I think it might be a good idea to just ditch the California non-COVID projection and burn rates as it causes a few issues:
    
   1. There's the issue of us not having ventilator burn rates for non-COVID patients, so it's kind of awkward to have a `0` in that spot.
   2. There's no upper or lower bounds on it, so it doesn't fit very nicely into the range scheme
   3. The CHA burn rate estimates are pretty back of the envelope

In [10]:
config = set_config('../config/ca-vent')
restart = NoteCompose(configdir='../config/ca-vent', population='dict')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

**These are the burn rates**

In [11]:
burn_sheet = format_population(to_sheet(demand.demand_per_unit_map_dn_um.df))
burn_sheet

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format='0,000', read_only=True, row_end=1, …

The only thing not from the CHA memo is the ventilator burn rate, which comes from IHME. They simply estimate ICU demand as 27% of all COVID hospitalizations, and make the assumption that 90% of COVID ICU patients will need to go on vent, so (0.27)(0.9) = 0.243. 

The rest of the burn rates are derived from the CHA memo (and this model replicates the PPE projections in that memo). I'm a bit uncomfortable with the burn rates - when you scale them to the IHME projections, for example, it's about 2.5 million N95 masks *per day* - that's definitely unrealistic. They *are* pretty much straight from the horse's mouth, but it might be wise for us to be clear about the source so that, as an organization, we're a bit insulated from them.

Below are the population estimates we're using, and then also the resulting projections. I've added a range dimension, but not all patient projections contain ranges. In these cases I do the horrible thing of just giving one exponential on either side of the mean. If this assumption is not acceptable, I've also included a version without the range calculations. Note that these are all **daily totals** - we can't simply do a scalar multiply by the number of days we're projecting for since some of these are reusable and some aren't (namely the ventilators). 

First are the calculations with no ranges, using an upper bound (or if only one number is given, just that one number).

In [12]:
import ipywidgets as widgets
pop_sheet = format_population(to_sheet(population.population_pP_tr.df), round=True)
daily_sheet = format_population(to_sheet(demand.demand_by_pop_total_pn_tc.df), round=True)

widgets.VBox([pop_sheet, daily_sheet])

Next is the same model but with the added range dimension. Use the slider to adjust the confidence interval, and the projections will automatically update. Note how some of the models (with my extremely questionable lower/upper bound assumptions) give us some alarmingly high projections for ventilators.

In [13]:
import ipywidgets as widgets
from scipy.stats import norm
import math

def triangular(a,b,c):
    return math.sqrt( ((a*a + b*b + c*c) - a*b - a*c - b*c) / 18 )


epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
stdev = epi_df.apply(
    lambda row: triangular(
        row['Vent Mid'],
        row['Vent Low'], 
        row['Vent High']
    ), axis=1
)

epi_df['Vent Mean'] = epi_df.mean(axis=1)

epi_df['Vent SD'] = stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Vent EOQ'] = df['Vent Mean'] + (Z * df['Vent SD'])
    return df

def display_eoq(cr):
    # calculate the hospitalization EOQ
    epi = calc_eoq(epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_df.loc['CA Projected Non-COVID Patients', 'Vent EOQ'] = 0
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (demand.demand_by_pop_per_person_pn_uc.array.T * epi_df["Vent EOQ"].to_numpy().T).T
    preds_df.df.columns = [f"Vent EOQ at CR={cr}", 'N95 Masks', 'Surgical Masks', 'PAPR', 'Gowns', 'Coveralls', 'Gloves']
    preds_sheet = to_sheet(preds_df.df)
    display_population(preds_sheet, round=True)

slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
out = widgets.interactive_output(display_eoq, {'cr': slider})
widgets.VBox([slider, out])

And then this is a rough go at a Vuetify UI - I'll make this prettier and we can package the write-up nicely inside here once that's done.

In [6]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Home']),
    v.Tab(children=['Projected Patients']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[
                    "Hospitalizations due to COVID at a peak surge were projected using several different models. \
                     From this figure, the number of patients requiring ICU care and an invasive ventilator using \
                     IHME figures. The remaining resources were projected using CHA burn rate assumptions." 
                ])
            ]),
            slider, out
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Population"]),
                v.CardSubtitle(children=["Projected Patients"]),
                pop_sheet
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand"]),
                burn_sheet
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['Home']), Tab(children=['Projected Patients']), Tab(children=['Burn Rates']), Tab…